In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# import the trainingset
data = pd.read_csv('letter-recognition.csv')  
data.head(5)

,T,2,8,3,5,1,8.1,13,0,6,6.1,10,8.2,0.1,8.3,0.2,8.4
0,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
1,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
2,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
3,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
4,S,4,11,5,8,3,8,8,6,9,5,6,6,0,8,9,7


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KernelDensity as KDe
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
# prepare the trainging data and test data
X = data[data.columns[:-1]].values
y = data['target'].values
cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [ ]:
# cross validation
def cross_validation(model):
    in_sample_error = []
    out_sample_error = []
    err_zscore = []

    for train_index, val_index in cv.split(X):
        Xtrain, Xval = X[train_index], X[val_index]
        ytrain, yval = y[train_index], y[val_index]
        model.fit(Xtrain,ytrain)

        pred_train,pred_val = model.predict(Xtrain),model.predict(Xval)
        pred = np.concatenate((pred_train, pred_val), axis=None)
        true_label = np.concatenate((ytrain,yval),axis=None)
        error = np.square(pred - true_label)/true_label.shape[0]
        #error = pred - true_label
        err_zscore.append(stats.zscore(error).reshape(-1,1))
        in_sample_error.append(mean_squared_error(pred_train, ytrain, squared=True))
        out_sample_error.append(mean_squared_error(pred_val, yval, squared=True))

    in_sample_error = np.array(in_sample_error)
    out_sample_error = np.array(out_sample_error)
    return in_sample_error,out_sample_error, err_zscore


* Random Forest

In [ ]:
RF = RandomForestRegressor(max_depth=4, random_state=0)
RF_in_sample_err,RF_out_sample_err, RF_zscore_err=cross_validation(RF)
# print(np.mean(RF_out_sample_err))
kde = KDe(bandwidth=1.0, kernel='gaussian')
kde.fit(RF_zscore_err)
lp = kde.score_samples(RF_zscore_err)
print(lp.shape)
sns.kdeplot(RF_zscore_err[:,0])